# Lesson: more tools for statistical and climatological analysis

Last week we learned how to open a NetCDF file (a data format which is very common in atmospheric applications), select variables, do simple statistics on them, and plot them. Today we are going to introduce some more data-crunching tools and we are going to learn how to make our plots more precise and informative.

## Temperature data  

Let's open the ERA-Interim 2m air temperature file (``ERA-Int-Monthly-2mTemp.nc``).

As usual, the file is available in the scratch folder (``/scratch/c707/c7071047/data/``) or on OLAT.

In [ ]:
# Display the plots in the notebook:
%matplotlib inline

In [ ]:
# Import the tools we are going to need today:
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xarray as xr  # netCDF library
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list
# Some defaults:
plt.rcParams['figure.figsize'] = (12, 5)  # Default plot size
np.set_printoptions(threshold=20)  # avoid to print very large arrays on screen
# The commands below are to ignore certain warnings.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ds = xr.open_dataset('./data/ERA-Int-Monthly-2mTemp.nc')
print(ds)

**Q: do you recall what ERA-Interim data is? Use your favorite search engine to get some basic information about it. What institution provides the data? Which time period is it available for? Where could you get the data if you need it for your upcoming thesis?**

**Try to explain in a few sentences how you think that the ``t2m`` variable available in this NetCDF file was generated. Is it OK to call these data "observations"? Why not? Reanalysis data often plays the role of "observations" when they are used to evaluate the output of climate models. Can you tell why?**

In [ ]:
# your answer here

When we do ``print(ds)``, as so-called "string representation" of the dataset is printed in the cell output. It contains enough information to answer the following questions:
- **what are the dimensions of the t2m variable?**
- **what is the spatial resolution of the ERA-Interim dataset?**
- **what is its temporal resolution?**
- **what it the data type of the various variables and coordinates available in the dataset? What are the differences between each data type?**

In [ ]:
# your answer here

## More control on plots

Let's compute the time average of the air temperature:

In [ ]:
t2_tavg = ds.t2m.mean(dim='time')
ax = plt.axes(projection=ccrs.Robinson())
t2_tavg.plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.coastlines(); ax.gridlines(); 

### Discrete levels

Smooth color tables like the above are nice and appealing, but the human eye is not trained to see such small differences in colors. For example. it would be quite difficult to tell which temperature does the coast of Peru have (above 280K? or below?). Sometimes, discrete levels are the way to go:  

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2_tavg.plot(ax=ax, transform=ccrs.PlateCarree(), levels=[240, 260, 280, 285, 290, 295, 300]) 
ax.coastlines(); ax.gridlines(); 

**Q: in which range can we place the temperature off the coast of Peru? If your eye still doesn't make the difference, how can we still be sure?**
**E: make a new plot, but this time set ``levels=12``.**

In [ ]:
# your answer here

### Color tables

**E: make a new variable called ``t2c_tavg``, which is ``t2_tavg`` converted to degrees celsius. Plot it.**

In [ ]:
t2c_tavg = t2_tavg - 273.15
ax = plt.axes(projection=ccrs.Robinson())
t2c_tavg.plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.coastlines(); ax.gridlines(); 

What happened to our plot? Note the location of the 0 on the colorbar and the automated choice of a new colorscale. Note also that the data range is mostly dictated by very cold temperatures in Antarctica. These automated choices are not allways meaningfull. Let's play around a little bit:

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2c_tavg.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='inferno', center=False, 
              vmin=-40, vmax=20, levels=7, cbar_kwargs={'label':'°C'}) 
ax.set_title('Average annual 2m air temperature, ERA-Interim 1979-2014')
ax.coastlines(); ax.gridlines(); 

**E: try to understand the role of each keyword by trying to use each of them separately. If you're still unsure, a look at xarray's [documentation](http://xarray.pydata.org/en/stable/generated/xarray.plot.pcolormesh.html) might be helpful.**

Note: the list of matplotlib's color tables can be found [here](http://www.matplotlib.org/examples/color/colormaps_reference.html).

In [ ]:
# your playground here

## Working with time series

**E: print the dataset's time coordinate. Which period do the data cover?**

In [ ]:
# your answer here

The selection tools we learned about last week also apply to time series. They are quite clever and very flexible:

In [ ]:
t2 = ds.t2m
t2.sel(time='2008')

**Q: what did we just do? Does that make sense to you? Try to understand the ouptut of the following commands:**
- t2.sel(time='2008-02')
- t2.sel(time='2008/02')
- t2.sel(time=slice('2008', '2012'))

In [ ]:
# Your answer here

### Time series of globally averaged fields 

If you remember last week, you should understand well what the following command will do:

In [ ]:
ts_t2_c = t2.mean(dim=['longitude','latitude']) - 273.15  # convert into Celsius
ts_t2_c.plot();

Logically, the global average temperature on Earth would be: 

In [ ]:
ts_t2_c.mean(dim='time')

But... wait? This is way colder than expected! (**Q: do you remember what is the average air temperature at the Earth surface?**). Indeed, we forgot to take the latitudinal effect into account.

Let's use a little bit of xarray magic to get things sorted. First, we can recall that averaging zonally (over the longitudes) is authorized, since all meridional slices have the same weight:

In [ ]:
zonal_mean_t2_c = t2.mean(dim='longitude') - 273.15  # convert into Celsius

**Q: what are the new dimensions of ``zonal_mean_t2_c``?**

So far so good. Now we are going to compute our latitudinal weight like last week:

In [ ]:
weight = np.cos(np.deg2rad(ds.latitude))
weight = weight / weight.sum()

Last week we used this weight to multiply it with an array of the same length (a zonal average without temporal dimension). Today we will use a  bit of magic, called "broadcasting" in the numpy jargon:

In [ ]:
weighted_zonal_mean_t2_c = zonal_mean_t2_c * weight

What did we just do? We multiplied a 2d array with a 1d array. This operation is not defined a priori, but xarray decided for us that what we *really* wanted to do is to multiply this 1d array with all 432 time slices we have in our array. Xarray knows how to do this because both our arrays have a matching dimension and associated coordinate variable. Thanks to this little trick, making a weighted average in 2 dimensions is as easy as it is in 1 dimension: 

In [ ]:
weighted_ts_t2_c = weighted_zonal_mean_t2_c.sum(dim='latitude')

**E: make a plot of this time series and compute its average. Does the global average now better fits to what we said in the lecture? If not, why?**

In [ ]:
# your answer here

Note that the operations above were only valid because averaging is a linear operation, i.e. commutative. For any field $A$, the equality $\left[ \overline{A} \right] = \overline{\left[A \right]}$ is always true.

### Resample time series data

Resampling is the operation of changing the *sampling* of the data, i.e. the frequency at which it is sampled. One of the most meaningful way to resample is to do an average, for example over the year:

In [ ]:
tsa_t2_c = weighted_ts_t2_c.resample(dim='time', freq='AS')

**E: check that ``tsa_t2_c`` is what you think it is. What could the ``'AS'`` mean? Try ``'A'`` for a change, and note the difference.**

Note that averaging is [not the only way](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.resample.html) available to resample data. **Try ``how='std'`` and  ``how='max'``, too.**

In [ ]:
# your answer here

**E: describe the features of the yearly average plot. Does it surprise you, or is it the way you thought it would be? Compute the standard deviation of this time series (tip: .std())** 

In [ ]:
# your answer here

### Compute the monthly climatographies (or annual cycle)

Another way to look at time series data is to average them according to the time of year to study the annual cycle. This is done with the ``.groupby()`` method:

In [ ]:
tsm_t2_c = weighted_ts_t2_c.groupby('time.month').mean()

**E: check that ``tsm_t2_c`` is what you think it is. What is the new dimension of the data? Plot it. Can you explain what you see?**

In [ ]:
# your answer here

### Averages and anomalies 

A very common way to decompose time-series is to normalize them. Let $A$ be any field, function of time $t$. Then:

$$A(t) = \overline{A} + A'(t)$$

Where $A'$ is called the fluctuating component of $A$, $A'(t)$ being the anomaly of $A$ at the time $t$. Often the variable t is omitted from the notation, i.e. one writes $A = \overline{A} + A'$

**E: compute the temperature anomaly for the year 1997 with respect to the 1979-2014 average.**

In [ ]:
# your answer here

**E: by noting that the functions applied to 1d data above also apply to 3d data, compute the temperature anomaly for the month of January 2010 with respect to the 1979-2014 January average.**

In [ ]:
# your answer here

## Selecting specific areas of our data

As for the time above, one can select slices out of the data:

In [ ]:
t2c_reg = t2c_tavg.sel(longitude=slice(-20, 20))

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree()) # Note that I changed the projection
t2c_reg.plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.add_feature(cartopy.feature.BORDERS); # What does this command do? 
ax.coastlines();

**E: create a new "t2c_reg" variable which is a subset of t2c_tavg between the longitudes (-20, 60) and the latitudes (40, -40). Plot the result.** (*hint: yes, I wrote (40, -40) and not (-40, 40)*)

In [ ]:
# your answer here

### Selection based on a condition

What if we are interested into air temperature on land only, and want to remove the oceans from our analyses? For this we are going to have to "mask out" the oceans grid points. First, we will need to open the "invariant: file: 

In [ ]:
nc_inv = xr.open_dataset('./data/ERA-Int-Invariant.nc')
nc_inv

**E: explore this new data file (variables, units). Plot the two variables.**

In [ ]:
# your answer here

OK, so "1" is land, "0" is ocean. We are going to use this information to mask out the values from the ocean:

In [ ]:
masked_t2_avg = t2c_tavg.where(nc_inv.lsm == 1)

**E: Plot this new variable. Compute its zonal average and plot it!**

In [ ]:
# your answer here

## Dimensional juggling! 

I am now going to apply a series of commands to our data. Let's see if you can follow each step:

In [ ]:
t2_m_reg = ds.t2m.sel(longitude=slice(-20, 60), latitude=slice(40, -40)).groupby('time.month').mean(dim='time') - 273.15
t2_m_reg_z = t2_m_reg.mean(dim='longitude')
t2_m_reg_z.T.plot();

Can you follow what I did? If not, decompose each step and see if you can follow the operations one by one.

The plot above is called a [Hovmöller](https://en.wikipedia.org/wiki/Hovm%C3%B6ller_diagram), used very often in climatology. **Q: Can you describe its features?**


*Note that the first line of code above is considered "bad practice" by programmers: lines that are too long are difficult to read and they are more likely to contain bugs. It is therefore NOT recommended to follow this example.* 

## Contour plots

Reconsider the Africa plot and the Hovmöller plot above. Both are quite "pixellized" (the first one because at regional scale, the coarse spatial resolution of ERA-Interim data becomes visible). Xarray's method of choice to display 2d data is to represent it as if it was an "image". It's ok most of the time, but sometimes you'd like to plot more contoured data. For example: 

In [ ]:
t2_m_reg_z.T.plot.contourf(levels=np.linspace(10, 30, 11));

## What next? 

**Copy the next exercise to your directory and [open it](06_Exercises_TemperaturePrecipitation.ipynb).** You can download it from OLAT into your folder, or you can download it from github:

    $ wget https://raw.githubusercontent.com/fmaussion/teaching/master/current/06_Exercises_TemperaturePrecipitation.ipynb